## Basit Taş Kağıt Makas Oyunu
### Bu kodda bilgisayar tamamen rastgele oynuyor. Birkaç el oynayıp sonuçları gözlemleyelim.

In [ ]:
import random

# Olası hamleler
moves = ['taş', 'kağıt', 'makas']

# Skorlar
user_score = 0
computer_score = 0

# Kazanma kuralları
def determine_winner(user_move, comp_move):
    if user_move == comp_move:
        return 'beraberlik'
    elif (user_move == 'taş' and comp_move == 'makas') or \
         (user_move == 'kağıt' and comp_move == 'taş') or \
         (user_move == 'makas' and comp_move == 'kağıt'):
        return 'kullanıcı'
    else:
        return 'bilgisayar'

# Oyun döngüsü
for round in range(1, 10):
    print(f"\n--- {round}. Tur ---")
    user_move = input("Hamleni gir (taş/kağıt/makas): ").lower()
    
    if user_move not in moves:
        print("Geçersiz hamle, tekrar deneyin.")
        continue
    
    comp_move = random.choice(moves)
    print(f"Bilgisayarın hamlesi: {comp_move}")
    
    result = determine_winner(user_move, comp_move)
    
    if result == 'kullanıcı':
        print("Bu turu sen kazandın!")
        user_score += 1
    elif result == 'bilgisayar':
        print("Bu turu bilgisayar kazandı!")
        computer_score += 1
    else:
        print("Beraberlik!")

print(f"\n🎯 Son Skor - Kullanıcı: {user_score} | Bilgisayar: {computer_score}")



## Markov Modeli ile Taş-Kâğıt-Makas
### Kullanıcının her hamlesinden sonra, bir sonraki hamlesi kaydedilir.

### Geçmiş hamle istatistiklerine göre, kullanıcı bir hamleden sonra en çok ne yapmışsa, onu yapacağı varsayılır.

### Bilgisayar da bu tahmine göre ona karşı gelen hamleyi seçer.

In [ ]:
import random
from collections import defaultdict

moves = ['taş', 'kağıt', 'makas']
beats = {'taş': 'kağıt', 'kağıt': 'makas', 'makas': 'taş'}

# 2. derece Markov: (prev2, prev1) -> sonraki hamle sayıları
transition_counts = defaultdict(lambda: defaultdict(int))

user_history = []
user_score = 0
computer_score = 0

def determine_winner(user_move, comp_move):
    if user_move == comp_move:
        return 'beraberlik'
    elif (user_move == 'taş' and comp_move == 'makas') or \
         (user_move == 'kağıt' and comp_move == 'taş') or \
         (user_move == 'makas' and comp_move == 'kağıt'):
        return 'kullanıcı'
    else:
        return 'bilgisayar'

def predict_next_move(prev2, prev1):
    key = (prev2, prev1)
    if key not in transition_counts:
        return random.choice(moves)
    next_moves = transition_counts[key]
    if not next_moves:
        return random.choice(moves)
    # En olası hareketi tahmin et
    predicted = max(next_moves, key=next_moves.get)
    return predicted

for round in range(1, 11):
    print(f"\n--- {round}. Tur ---")
    user_move = input("Hamleni gir (taş/kağıt/makas): ").lower()

    if user_move not in moves:
        print("Geçersiz hamle.")
        continue

    # Tahmin ve seçim
    if len(user_history) < 2:
        comp_move = random.choice(moves)
    else:
        predicted_user_move = predict_next_move(user_history[-2], user_history[-1])
        comp_move = beats[predicted_user_move]
    
    print(f"Senin hamlen: {user_move} | Bilgisayarın hamlesi: {comp_move}")
    
    result = determine_winner(user_move, comp_move)
    if result == 'kullanıcı':
        print("Bu turu sen kazandın!")
        user_score += 1
    elif result == 'bilgisayar':
        print("Bu turu bilgisayar kazandı!")
        computer_score += 1
    else:
        print("Beraberlik!")

    # Markov modelini güncelle
    if len(user_history) >= 2:
        key = (user_history[-2], user_history[-1])
        transition_counts[key][user_move] += 1
    user_history.append(user_move)

print(f"\n📊 Son Skor - Kullanıcı: {user_score} | Bilgisayar: {computer_score}")


## Q-learning ile Taş-Kağıt-Makas Oyunu (Temel)
### Durum (state): Kullanıcının son hamlesi (örneğin: taş, kağıt, makas)

### Aksiyon (action): Bilgisayarın oynayacağı hamle (taş, kağıt, makas)

### Ödül (reward): Kazanma = +1, Beraberlik = 0, Kaybetme = -1

In [ ]:
import random

moves = ['taş', 'kağıt', 'makas']
beats = {'taş': 'makas', 'kağıt': 'taş', 'makas': 'kağıt'}

# Q-Tablosu: state (son kullanıcı hamlesi) -> action (bilgisayar hamlesi) değerleri
Q = {move: {a: 0 for a in moves} for move in moves}

alpha = 0.1    # öğrenme hızı
gamma = 0.9    # gelecekteki ödüllerin indirgeme faktörü
epsilon = 0.2  # keşif oranı

def determine_winner(user_move, comp_move):
    if user_move == comp_move:
        return 0  # beraberlik
    elif beats[comp_move] == user_move:
        return 1  # bilgisayar kazandı
    else:
        return -1 # kullanıcı kazandı

def choose_action(state):
    if random.random() < epsilon:
        return random.choice(moves)  # keşif
    else:
        # En yüksek Q değerine sahip hareketi seç
        max_q = max(Q[state].values())
        actions_with_max_q = [a for a, q in Q[state].items() if q == max_q]
        return random.choice(actions_with_max_q)

user_history = []
user_score = 0
computer_score = 0

print("Taş-Kağıt-Makas Q-learning oyunu başlıyor!")

for round in range(1, 21):
    print(f"\n--- {round}. Tur ---")
    user_move = input("Hamleni gir (taş/kağıt/makas): ").lower()
    if user_move not in moves:
        print("Geçersiz hamle.")
        continue
    
    # Son kullanıcı hamlesine göre durum belirle
    if not user_history:
        state = random.choice(moves)  # Başlangıç için rastgele durum
    else:
        state = user_history[-1]
    
    comp_move = choose_action(state)
    print(f"Senin hamlen: {user_move} | Bilgisayarın hamlesi: {comp_move}")
    
    reward = determine_winner(user_move, comp_move)
    if reward == 1:
        print("Bilgisayar kazandı!")
        computer_score += 1
    elif reward == -1:
        print("Sen kazandın!")
        user_score += 1
    else:
        print("Beraberlik!")

    # Q-Tablosunu güncelle
    if user_history:
        old_state = user_history[-1]
        old_action = comp_move
        # Bir sonraki durum
        next_state = user_move
        max_future_q = max(Q[next_state].values())
        Q[old_state][old_action] = Q[old_state][old_action] + alpha * (reward + gamma * max_future_q - Q[old_state][old_action])

    user_history.append(user_move)

print(f"\n📊 Son Skor - Kullanıcı: {user_score} | Bilgisayar: {computer_score}")


## İki AI Ajanıyla Q-learning Taş-Kağıt-Makas Oyunu
### Durum (state): Rakibin önceki hamlesi (taş, kağıt veya makas)
### İlk hamle için özel durum: "Başlangıç" (rakibin hamlesi yok)
### Aksiyon (action): Oyuncunun oynayabileceği hamleler: taş, kağıt, makas
### Ödül (reward): Kazanma = +1 Beraberlik = 0 Kaybetme = -1

### Her bir AI, rakibin son hamlesini gözlemleyerek hangi hamleyi yaparsa kazanma şansının yüksek olduğunu Q-learning ile öğrenir.

In [9]:
import random
import numpy as np

moves = ['taş', 'kağıt', 'makas']
beats = {'taş': 'makas', 'kağıt': 'taş', 'makas': 'kağıt'}

# Q-learning parametreleri
alpha = 0.1    # öğrenme hızı
gamma = 0.9    # indirim faktörü
epsilon = 0.1  # keşif oranı
episodes = 500

# Q tabloları (state=son hamle, action=hamle)
Q1 = {move: np.zeros(len(moves)) for move in moves + ['Başlangıç']}
Q2 = {move: np.zeros(len(moves)) for move in moves + ['Başlangıç']}

def choose_action(Q, state):
    if random.random() < epsilon:
        return random.choice(range(len(moves)))
    else:
        return np.argmax(Q[state])

def reward(p1_move, p2_move):
    if p1_move == p2_move:
        return 0
    elif beats[p1_move] == p2_move:
        return 1
    else:
        return -1

# Oyun sonucu skorları
ai1_score = 0
ai2_score = 0
draw_count = 0

state1 = 'Başlangıç'
state2 = 'Başlangıç'

for episode in range(1, episodes + 1):
    action1 = choose_action(Q1, state1)
    action2 = choose_action(Q2, state2)

    move1 = moves[action1]
    move2 = moves[action2]

    r1 = reward(move1, move2)
    r2 = reward(move2, move1)  # tersi

    # Q-table güncelleme
    next_state1 = move2
    next_state2 = move1

    Q1[state1][action1] = Q1[state1][action1] + alpha * (r1 + gamma * max(Q1[next_state1]) - Q1[state1][action1])
    Q2[state2][action2] = Q2[state2][action2] + alpha * (r2 + gamma * max(Q2[next_state2]) - Q2[state2][action2])

    state1 = next_state1
    state2 = next_state2

    # Skor güncelleme
    if r1 == 1:
        ai1_score += 1
    elif r1 == -1:
        ai2_score += 1
    else:
        draw_count += 1

    if episode % 100 == 0:
        print(f"--- Tur {episode} ---")
        print(f"AI_1 Skor: {ai1_score}, AI_2 Skor: {ai2_score}, Beraberlik: {draw_count}")

print("\n📊 Oyun Sonucu:")
print(f"AI_1 kazandığı tur sayısı: {ai1_score}")
print(f"AI_2 kazandığı tur sayısı: {ai2_score}")
print(f"Beraberlik sayısı: {draw_count}")

if ai1_score > ai2_score:
    print("Kazanan: AI_1")
elif ai2_score > ai1_score:
    print("Kazanan: AI_2")
else:
    print("Oyun berabere bitti!")


--- Tur 100 ---
AI_1 Skor: 24, AI_2 Skor: 30, Beraberlik: 46
--- Tur 200 ---
AI_1 Skor: 67, AI_2 Skor: 60, Beraberlik: 73
--- Tur 300 ---
AI_1 Skor: 94, AI_2 Skor: 82, Beraberlik: 124
--- Tur 400 ---
AI_1 Skor: 135, AI_2 Skor: 126, Beraberlik: 139
--- Tur 500 ---
AI_1 Skor: 167, AI_2 Skor: 172, Beraberlik: 161

📊 Oyun Sonucu:
AI_1 kazandığı tur sayısı: 167
AI_2 kazandığı tur sayısı: 172
Beraberlik sayısı: 161
Kazanan: AI_2


## Nash Q-learning ile İki AI Ajanının Taş-Kağıt-Makas Oyunu  
### Durum (state): Rakibin önceki hamlesi (taş, kağıt veya makas) veya "Başlangıç"  
### Aksiyon (action): Oyuncunun oynayabileceği hamleler: taş, kağıt, makas  
### Ödül (reward): Kazanma = +1, Beraberlik = 0, Kaybetme = -1  

### Özet:  
- Her turda iki AI ajanı, mevcut durumdaki Q-matrislerine göre oyun matrisini oluşturur.  
- Nashpy kütüphanesi kullanılarak bu matrisler için Nash dengesi (karşılıklı en iyi strateji karışımı) hesaplanır.  
- Ajanlar, Nash dengesinden elde edilen karma stratejilere göre hareket seçer.  
- Q tabloları, ödül ve gelecekteki Nash dengesine göre beklenen getirilerle güncellenir.  
- Böylece ajanlar sadece kendi ödüllerini değil, karşı tarafın stratejik davranışlarını da hesaba katarak öğrenir.  
- Bu yöntem, klasik Q-learning’den farklı olarak çok oyunculu oyunlarda denge stratejilerini öğrenmeyi hedefler.  


In [ ]:
!pip install nashpy



In [13]:
import numpy as np
import nashpy as nash

# Taş-kağıt-makas tanımı
moves = ['taş', 'kağıt', 'makas']
num_moves = len(moves)

# Parametreler
alpha = 0.1       # öğrenme hızı
gamma = 0.9       # indirim faktörü
episodes = 2000   # toplam oyun sayısı

# Q tabloları: durum 'Başlangıç' için 3x3 matrisler
Q1 = {'Başlangıç': np.zeros((num_moves, num_moves))}
Q2 = {'Başlangıç': np.zeros((num_moves, num_moves))}

def reward(p1_move, p2_move):
    if p1_move == p2_move:
        return 0, 0
    elif (p1_move == 'taş' and p2_move == 'makas') or \
         (p1_move == 'kağıt' and p2_move == 'taş') or \
         (p1_move == 'makas' and p2_move == 'kağıt'):
        return 1, -1
    else:
        return -1, 1

def get_nash_strategy(Q1_mat, Q2_mat):
    """
    Verilen iki Q matrisi için Nash dengesini hesaplar.
    """
    game = nash.Game(Q1_mat, Q2_mat)
    equilibria = list(game.support_enumeration())
    if equilibria:
        sigma1, sigma2 = equilibria[0]
    else:
        # Nash bulunamazsa eşit olasılık dağılımı
        sigma1 = np.ones(num_moves) / num_moves
        sigma2 = np.ones(num_moves) / num_moves
    return sigma1, sigma2

# Başlangıç durumu
state = 'Başlangıç'

# Kazanma sayacı
score_1 = 0
score_2 = 0
draws = 0

for episode in range(1, episodes+1):
    # Durum için Q tablosu yoksa oluştur
    if state not in Q1:
        Q1[state] = np.zeros((num_moves, num_moves))
        Q2[state] = np.zeros((num_moves, num_moves))

    Q1_state = Q1[state]
    Q2_state = Q2[state]

    # Nash stratejilerini bul
    sigma1, sigma2 = get_nash_strategy(Q1_state, Q2_state)

    # Stratejilere göre hamle seç
    action1 = np.random.choice(num_moves, p=sigma1)
    action2 = np.random.choice(num_moves, p=sigma2)

    move1 = moves[action1]
    move2 = moves[action2]

    # Ödüller
    r1, r2 = reward(move1, move2)

    # Gelecek durum (örnek basit: rakibin son hamlesi)
    next_state = move2
    if next_state not in Q1:
        Q1[next_state] = np.zeros((num_moves, num_moves))
        Q2[next_state] = np.zeros((num_moves, num_moves))

    # Gelecek durumun Nash stratejileri
    sigma1_next, sigma2_next = get_nash_strategy(Q1[next_state], Q2[next_state])

    # Gelecek durumun beklenen ödülleri
    expected_q1 = 0
    expected_q2 = 0
    for a1 in range(num_moves):
        for a2 in range(num_moves):
            expected_q1 += sigma1_next[a1] * Q1[next_state][a1, a2] * sigma2_next[a2]
            expected_q2 += sigma1_next[a1] * Q2[next_state][a1, a2] * sigma2_next[a2]

    # Q güncelleme
    Q1[state][action1, action2] = (1 - alpha) * Q1[state][action1, action2] + alpha * (r1 + gamma * expected_q1)
    Q2[state][action1, action2] = (1 - alpha) * Q2[state][action1, action2] + alpha * (r2 + gamma * expected_q2)

    # Durumu güncelle
    state = next_state

    # Skorları güncelle
    if r1 == 1:
        score_1 += 1
    elif r2 == 1:
        score_2 += 1
    else:
        draws += 1

    # Her 200 turda durumu yazdır
    if episode % 200 == 0:
        print(f"--- Tur {episode} ---")
        print(f"AI_1 hamlesi: {move1} | AI_2 hamlesi: {move2}")
        print(f"Skor: AI_1 = {score_1}, AI_2 = {score_2}, Beraberlik = {draws}\n")

# Final skoru
print("=== Oyun Sonu ===")
print(f"AI_1 kazandı: {score_1}")
print(f"AI_2 kazandı: {score_2}")
print(f"Beraberlikler: {draws}")


--- Tur 200 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 200

--- Tur 400 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 400

--- Tur 600 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 600

--- Tur 800 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 800

--- Tur 1000 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 1000

--- Tur 1200 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 1200

--- Tur 1400 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 1400

--- Tur 1600 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 1600

--- Tur 1800 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 1800

--- Tur 2000 ---
AI_1 hamlesi: taş | AI_2 hamlesi: taş
Skor: AI_1 = 0, AI_2 = 0, Beraberlik = 2000

=== Oyun